# 匯入套件

In [1]:
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup as bs
import json
import ast
import re
import time
from collections import defaultdict

# 網頁檢索

In [2]:
def request_all():
    res = requests.get('https://www.beautyclinic.com.tw/page/doctors')
    raw_data = etree.HTML(res.content.decode('utf-8'))
    get_detail(raw_data)

# 取得資訊

In [3]:
def get_detail(raw_data):
    getfromjs = raw_data.xpath("//*[@id='content']/script/text()")
    
    #取得診所資訊
    addr_data={}
    for i in raw_data.xpath("//div[contains(@class,'col_half')]//li")[:-1]:
        key_ = i.xpath('text()')[0].split('/')[0].strip()
        temp = i.xpath('div/text()')
        val_ = {'cli_phone' : temp[0],'cli_addr' : temp[1]}
        tempdict={key_:val_}
        addr_data.update(tempdict)
    #取得醫生資訊
    data_to_df = list()
    for each in getfromjs[0].split('docsRawData.push(')[1:]:
        try:
            data = eval(each.split(');')[0])
            data['featured_img'] = 'https://www.beautyclinic.com.tw'+data['featured_img']
            default=defaultdict(list)
            for defaultidx in data['locations']:
                for key, value in defaultidx.items():
                    default[key].append(value)
                data['locations']=dict(default)['name']  
            raw_data = bs(data['intro'],'html5')
            data['intro'] = raw_data.text
            for loc_idx in range(len(data['locations'])):
                if 'cli_phone' not in data:
                    data.update(addr_data[data['locations'][loc_idx]])
                else:
                    data['cli_phone']=data['cli_phone']+','+''.join(addr_data[data['locations'][loc_idx]]['cli_phone'])
                    data['cli_addr']=data['cli_addr']+','+''.join(addr_data[data['locations'][loc_idx]]['cli_addr'])
            data['locations'] = ','.join(data['locations'])
            data_to_df.append(data)
        except SyntaxError:
            pass
        except Exception as err:
            print('Error: ',err)
    to_csv(data_to_df)

# to_csv

In [4]:
def to_csv(data_to_df):
    FILE_NAME = "佳醫美人"
    today = time.strftime("%Y%m%d")
    df = pd.DataFrame(data_to_df)
    df.rename(columns={'name':'doc_name',"featured_img":"doc_pic",'locations':'countries','intro':'doc_edu_exp',"sp":"doc_prof"}, inplace=True)
    df.drop(['id', 'job','ename'], axis=1, inplace=True)
    df.to_csv(f"{FILE_NAME}-{today}.csv",index=False)

# 執行程式

In [5]:
request_all()